In [83]:
import pandas as pd             #pandas for using dataframe and reading csv file(s)
import numpy as np              #numpy for vector operations and basic maths
import matplotlib.pyplot as plt #for plotting
%matplotlib inline              
import seaborn as sns           #for making plots
from haversine import haversine #for working with latitudinal and longitudinal data points
import math                     #for basic math operations
import warnings
from pandas.plotting import parallel_coordinates #for multivariate plots
warnings.filterwarnings('ignore') #ignore deprecation warnings
import folium
from folium.plugins import HeatMap

In [84]:
#importing data
combined_data = pd.read_csv('/Users/anthonymoubarak/Desktop/511 Project/Combined_df_teg.csv')

In [108]:
# Cluster data 

combined_data = pd.read_csv('/Users/anthonymoubarak/Desktop/511 Project/Combined_df_teg.csv')
clustered_data = combined_data.drop(['Unnamed: 0', 'Unnamed: 1'], axis=1)
combined_data['Member.type'] = combined_data['Member.type'].str.lower()

In [109]:
from statistics import mode
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.metrics import accuracy_score
from math import radians

data_model = combined_data #.sample(frac=0.1).reset_index(drop=True)

data_model['start_lat'] = data_model['start_lat'].apply(radians)
data_model['start_lng'] = data_model['start_lng'].apply(radians)
data_model['end_lat'] = data_model['end_lat'].apply(radians)
data_model['end_lng'] = data_model['end_lng'].apply(radians)

X = data_model[['start_lat', 'start_lng', 'end_lat', 'end_lng']].values
y = data_model['Duration'].values

bestK = KMeans(n_clusters=3, init='k-means++', random_state=42)
labels4 = bestK.fit_predict(X)
data_model['kmeans_labels'] = labels4

In [110]:
# Convert back to degrees


data_model['start_lat'] = (data_model['start_lat']*180)/math.pi
data_model['start_lng'] = (data_model['start_lng']*180)/math.pi
data_model['end_lat'] = (data_model['end_lat']*180)/math.pi
data_model['end_lng'] = (data_model['end_lng']*180)/math.pi

In [116]:
def show_fmaps2(data, path=1):
    """function to generate map and add the pick up and drop coordinates
    1. Path = 1 : Join pickup (blue) and drop(red) using a straight line
    """
    map_1 = folium.Map(location=[39.13,-77.4], zoom_start=9) # manually added centre
    data_reduced = data.sample(frac=0.001).reset_index(drop=True)
    
    for i in range(data_reduced.shape[0]):
        try:
            pick_long = data_reduced.loc[data_reduced.index ==i]['start_lng'].values[0]
            pick_lat = data_reduced.loc[data_reduced.index ==i]['start_lat'].values[0]
            dest_long = data_reduced.loc[data_reduced.index ==i]['end_lng'].values[0]
            dest_lat = data_reduced.loc[data_reduced.index ==i]['end_lat'].values[0]
        
            
            if data_reduced['kmeans_labels'][i] == 0:
                folium.Marker([pick_lat, pick_long], icon=folium.Icon(color='red')).add_to(map_1)

            elif data_reduced['kmeans_labels'][i] == 1:
                folium.Marker([pick_lat, pick_long], icon=folium.Icon(color='blue')).add_to(map_1)
            elif data_reduced['kmeans_labels'][i] == 2: 
                folium.Marker([pick_lat, pick_long], icon=folium.Icon(color='blue')).add_to(map_1)

        except:
            continue

    folium.TileLayer('Stamen Terrain').add_to(map_1)
    folium.TileLayer('Stamen Toner').add_to(map_1)
    folium.TileLayer('Stamen Water Color').add_to(map_1)
    folium.TileLayer('cartodbpositron').add_to(map_1)
    folium.TileLayer('cartodbdark_matter').add_to(map_1)
    folium.LayerControl().add_to(map_1)
       
    return map_1

In [117]:
x = show_fmaps2(data_model)
x.save('test.html')